In [892]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from itertools import combinations, permutations
from collections import defaultdict
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import networkx as nx

%matplotlib inline

In [893]:
def turn_to_url(season):
    return 'http://survivor.wikia.com/wiki/Survivor:_' + season.replace(' ','_')

def process(string):
    stripped = string.strip()
    return re.sub('[0-9]+', '', stripped)

seasons = ['Borneo', 'The Australian Outback', 'Africa', 'Marquesas', 'Thailand', 'The Amazon', 
           'Pearl Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala', 'Panama', 
           'Cook Islands', 'Fiji', 'China', 'Micronesia', 'Gabon', 'Tocantins', 'Samoa', 
           'Heroes vs. Villains', 'Nicaragua', 'Redemption Island', 'South Pacific',
           'One World', 'Philippines', 'Caramoan', 'Blood vs. Water', 'Cagayan', 'San Juan del Sur',
           'Worlds Apart', 'Cambodia', 'Kaôh Rōng', 'Millennials vs. Gen X',
           'Game Changers', 'Heroes vs. Healers vs. Hustlers', 'Ghost Island','David vs. Goliath']

In [894]:
#caramoan = extract_voting_table_as_df(turn_to_url(seasons[25]))

#Cambodia
#Game Changers
#Heroes vs. Healers vs. Hustlers

#test = 
#extract_voting_table_as_df(turn_to_url('Heroes vs. Healers vs. Hustlers'))
#test2 = test.replace(to_replace=' ', value = '_',regex=True)#'Laura M.','Laura_M')
#test3 = test2.replace('.','')#to_replace='.', value = '',regex=True)
#test3

In [895]:
#plot_network('Heroes vs. Healers vs. Hustlers')

In [896]:
def calculate_alliance_index(series):
    """
        Calculate the alliance index which is defined as:
            alliance((i,j),t) = (1/t_{i,j})*sum_0^t(affiliation((i,j))
        where (i,j) is a contestant pair and t is the voting round and t_{i,j} is the
        number of times (i,j) appeared at voting round together
        
        alliance((i,j),t) is basically the percentage of the time that
        i and j agreed on who to vote out
        
        alliance(i,j) = alliance((i,j),T), where T is the last elimination voting round,
        can be used to summarize the relationship of (i,j) for the season
    
        note that alliance(i,j) does not consider the number of tribal councils
        that i and j  experienced together. to take this into account, we can filter 
        the pairings to find significant relationships (for example, consider only pairs 
        who experienced 5 voting rounds together)
        
        input : series - pandas series
        output : alliance index series
    """
    series = series.dropna()
    out = []
    for i in range(len(series)):
        out.append(series[:i+1].mean())
    return pd.Series(out, index=series.index)

In [897]:
def return_alliance_df_with_filter(bootdd,dd, k):
    """
        Return data frame of alliance index for every voting round
        input:
            dd - data frame of voting table
            k - filter for the number of voting rounds together
    """
    adf,bootdd = return_affiliation_df(bootdd,dd)
    return adf[adf.count(axis=1)>k].apply(calculate_alliance_index,axis=1).fillna(method='ffill',axis=1),bootdd

In [898]:
def find_str(tup,string):
    return (tup[0] == string) or (tup[1] == string)

def simplify(tup, selected):
    if selected == tup[0]:
        return tup[1]
    else:
        return tup[0]

def plot_contestant_top_with_filter(dd, selected, season_name, m=5, k=5):
    """
        Plot top m contestants in terms of alliance_index with selected
        input:
            dd - data frame of voting table
            selected - contestant that you are focusing on
            m - top contestants to consider
            k - filter for number of voting rounds
    """
    filtered = return_alliance_df_with_filter(dd, k)
    with_selected = filtered[[find_str(i,selected) for i in filtered.index]].sort_values(len(filtered.columns)-1,ascending=False)
    top_m = with_selected.iloc[:m]
    top_m = top_m.reset_index()
    top_m['index'] = top_m['index'].apply(simplify,args=(selected,))
    top_m = top_m.set_index('index')
    if len(top_m.index) != 0:
        top_m.T.plot(style='o-', ms=8).legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title("{}'s top {} relationships (>{} voting rounds together)".format(selected, m, k))
    plt.ylabel('Alliance index')
    plt.xlabel('Voting round $t$')
    plt.xticks(list(top_m.columns))
    plt.yticks(np.arange(0,1.1,.1))
    plt.xlim(-1,len(top_m.columns))
    plt.ylim(-0.1,1.1)
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    plt.savefig('{}_{}.png'.format(season_name, selected), bbox_inches='tight')
    plt.close()
    
def return_df_contestant_top_with_filter(dd, selected, m, k=1):
    """
        return df of the plot above
    """
    filtered = return_alliance_df_with_filter(dd, k)
    with_selected = filtered[[find_str(i,selected) for i in filtered.index]].sort_values(len(filtered.columns)-1,ascending=False)
    top_m = with_selected.iloc[:m]
    return top_m

In [899]:
#plot_contestant_top_with_filter(caramoan, "Dawn", 'Caramoan', 5, k=5)

In [900]:
def return_top_m_relationships(bootdd,dd, m, k):
    """
        return the top m relationships of the season (measured via final alliance index)
        filtered by k times together in a voting round
    """
    adf,bootdd = return_alliance_df_with_filter(bootdd,dd, k)
    # sort values in decreasing alliance index
    adf = adf.sort_values(len(adf.columns)-1, ascending=False).iloc[:m,len(adf.columns)-1]
    # remove if 0 already
    return adf[adf!=0],bootdd

In [901]:
#top50 = return_top_m_relationships(caramoan, 50, 5)
#top50

In [902]:
#top50.index[0][0]


In [903]:
def plot_season_contestant_top_with_filter(season_name,selected, m=5, k=5):
    url = turn_to_url(season_name)
    dd,bootdd = extract_voting_table_as_df(url)
    filtered = return_alliance_df_with_filter(dd, k)
    with_selected = filtered[[find_str(i,selected) for i in filtered.index]].sort_values(len(filtered.columns)-1,ascending=False)
    top_m = with_selected.iloc[:m]
    top_m = top_m.reset_index()
    top_m['index'] = top_m['index'].apply(simplify,args=(selected,))
    top_m = top_m.set_index('index')
    sns.set_style("darkgrid")
    top_m.T.plot(style='o-', ms=8).legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title("{}'s top {} relationships (>{} voting rounds together)".format(selected, m, k))
    plt.ylabel('Alliance index')
    plt.xlabel('Voting round $t$')
    plt.xticks(list(top_m.columns))
    plt.yticks(np.arange(0,1.1,.1))
    plt.xlim(-1,len(top_m.columns))
    plt.ylim(-0.1,1.1)
    fig = plt.gcf()
    fig.set_size_inches(12, 6)
    plt.savefig('{}_{}.png'.format(season_name, selected), bbox_inches='tight')
    plt.close()

In [904]:
#plot_season_contestant_top_with_filter('Gabon','Sugar', m=5, k=5)

In [905]:
def plot_network(season_name,m=50,k=3):
    url = turn_to_url(season_name)
    print(url)
    dd,bootdd = extract_voting_table_as_df(url)
    dd
    top_m,bootdd = return_top_m_relationships(bootdd,dd, m, k)
    
    alliance = create_alliance_score(dd.index, top_m)#, ids)
    
    #print(top_m)
    plt.close()
    sns.set_style("white")
    plt.figure(figsize=(7,7))
    G=nx.Graph()

    with_edge = []
    for i in dd.index:
        for tup in top_m.index:
            if i in tup:
                with_edge.append(i)
                break
    G.add_nodes_from(with_edge)

    goldenrod=(218./255, 165./255, 32./255)
    gray=(0.9,0.9,0.9)
    #print(return_alliance_df_with_filter(dd, k))
    for i in range(len(top_m.index)):
        G.add_edge(top_m.index[i][0], top_m.index[i][1], weight=top_m.values[i])
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G,pos,node_color = goldenrod,node_size=800)
    nx.draw_networkx_labels(G,pos)
    weights = [5*G[u][v]['weight'] for u,v in G.edges()]
    nx.draw_networkx_edges(G,pos,width=weights,alpha= 0.7,edge_color = 'green')
    plt.title('{} Castaway Relationships. Top {}, Filter >{}'.format(season_name, m, k))
    plt.tick_params(
        axis='both',        
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        left=False,         # ticks along the top edge are off
        labelbottom=False,
        labelleft=False) # labels along the bottom edge are off
    plt.savefig('{}_network_top_{}_filter_{}.png'.format(season_name,m,k))
    plt.close()

    return alliance,bootdd

In [906]:
def create_alliance_score(names,alliances):#,ids):
    #print (names)
    #print (alliances)
    scores = []
    for x in range(len(names)):
        sub = names[x]
        scrs = []
        for y in range(len(alliances)):
            tup = alliances.index[y]
            if sub in tup:
                scrs.append(alliances[y])
        if len(scrs) == 0: scores.append([sub,-1])       
        if len(scrs) > 0: scores.append([sub,np.mean(scrs)])
    return scores

In [907]:
def extract_voting_table_as_df(link):
    """
    Extract the voting history table as a pandas data frame.
    Input: link - url for the season page in Survivor wiki
    Output: dd - pandas dataframe of the voting history table
    """
    card = requests.get(link)
    soup = BeautifulSoup(card.content, 'lxml')
    
    global del_cols

    # use select() bec 2 css classes
    voting_table = soup.select('table.wikitable.article-table')[-1]
    #print(voting_table)
    # obtain the table cells of the voters (the leftmost column)
    names = voting_table.find_all('td', style = 'text-align: left;')
    names.extend(voting_table.find_all('td', style = 'text-align: left; white-space:nowrap;'))
    names.extend(voting_table.find_all('td', style = 'text-align: left; white-space: nowrap;'))
    #print(names)
    
    base = voting_table.find('a', class_ = "image image-thumbnail link-internal")
    boot_order = {'Name' : 'Boot'}
    ct=0
    for x in base.parent.parent.parent.contents:
        #print(ct,x)
        if str(type(x)) == "<class 'bs4.element.Tag'>":
            l = len(x.find_all('a',class_ = "image image-thumbnail link-internal"))
            if l == 1:
                #print("TAG")
                boot_order[ct] = str(x.find('a').next_sibling.next_sibling)
                ct+=1
            elif l == 0:
                if len(x.find_all('i')) == 1:
                    boot_order[ct] = str(x.find('i').text)
                    ct+=1
            else:
                print("ERROR")

    #print(boot_order)                    
        
    # obtain the names of the voters as strings
    contestants = []
    for voter in names:
        #if len(process(voter.text)) < 30:
        contestants.append(process(voter.text))
    #print(len(contestants))
    #print(contestants)

    no_vote = ['Quit', 'Evacuated']
    # we will convert this list into a pandas df
    season_list = []
    season_boot_list = []
    season_boot_list.append(boot_order)
    boot_score = {}
    lastvote = 0

    for voter in names:
        voter_dict = {'Name' : process(voter.text)}
        boot_score = {'Name' : process(voter.text)}
        inc = 0
        for voted in voter.find_next_siblings():
            if process(voted.text) in ['Jury Vote','']:
                break
            # some cell span more than one column
            # compensate for this by adding more '—'
            if voted.has_attr('colspan'):
                numcol = int(voted['colspan'])
                for i in range(numcol):  
                    voter_dict[inc] = '—'
                    inc += 1
                continue

            if (process(voted.text) in contestants) or (process(voted.text) == '—') or (process(voted.text) in no_vote):
                voter_dict[inc] = process(voted.text)
                #print(inc,boot_order,process(voted.text))
                if process(voted.text) == boot_order[inc]:
                    boot_score[inc] = 1
                elif process(voted.text) == '—':
                    boot_score[inc] = 0
                else:
                    boot_score[inc] = -1
                lastvote = max(lastvote,inc)
                    
            inc += 1
        #print('#####voterdict#######',voter_dict)
        season_list.append(voter_dict)
        season_boot_list.append(boot_score)
    #remove final tribal
    remove = []
    for z in season_boot_list[0].keys():
        if z != 'Name':
            if int(z) > lastvote: remove.append(z)
    #print (lastvote,remove)
    for z in remove:
        season_boot_list[0].pop(z,None)
    #print(season_list)
    
    # convert the list into a pd data frame and set as index the voters
    dd = pd.DataFrame(season_list).set_index('Name')
    bootdd = pd.DataFrame(season_boot_list).set_index('Name')
    #print("######BOOTDD##########\n",bootdd)
    # rearrange the columns of the data frame in increasing order
    dd = dd[sorted(dd.columns)]
    bootdd = bootdd[sorted(bootdd.columns)]
    
    #oddity correction for rock draws and other random events
    row = bootdd.iloc[0]
    if link == 'http://survivor.wikia.com/wiki/Survivor:_Marquesas':
        for i,val in enumerate(row):
            if val == 'Paschal':
                bootdd.iloc[:,i] = ['Paschal',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                print('found paschal')
        #bootdd['Paschal'] = 0
    if link == 'https://survivor.fandom.com/wiki/Survivor:_Caramoan':
        for i,val in enumerate(row):
            if val == 'Julia':
                bootdd.iloc[:,i] = ['Julia',1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0]
                print('found julia')
        #bootdd['Julia'] = [1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0]
    if link == 'https://survivor.fandom.com/wiki/Survivor:_Game_Changers':
        for i,val in enumerate(row):
            if val == 'Cirie':
                bootdd.iloc[:,i] = ['Cirie',0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                print('found cirie')
        #bootdd['Cirie'] = 0
    del_cols = []
    # remove columns which represent no voting: evacuated or quit
    for i in dd.columns:
        #print(dd[i].values)
        if ('Evacuated' in dd[i].values) or ('Quit' in dd[i].values):
            del_cols.append(i)
            continue
        for ind,j in enumerate(dd[i].values):
            if j in contestants:
                break
            if ind==len(dd[i])-1:     
                del_cols.append(i)
    #print('DELETE',del_cols)
    delcorrect =[['http://survivor.wikia.com/wiki/Survivor:_The_Australian_Outback',[7]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Africa',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Marquesas',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Palau',[0, 1, 12, 14]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Panama',[9]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Cook_Islands',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Gabon',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Blood_vs._Water',[5]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Worlds_Apart',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Cambodia',[5]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Kaôh_Rōng',[5,9,15]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Millennials_vs._Gen_X',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Heroes_vs._Healers_vs._Hustlers',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_Ghost_Island',[]]
    ,['http://survivor.wikia.com/wiki/Survivor:_David_vs._Goliath',[0,3]]]
    
    dd=dd.drop(del_cols,axis=1)
    
    #correct for deadlock
    for lnk,de in delcorrect:
        if link == lnk:
            del_cols = de
    
    for i in bootdd.columns:
        if ("Tie" in bootdd[i].values) or ("Deadlock" in bootdd[i].values) or ("Null" in bootdd[i].values):
            del_cols.append(i)

    #print('DELETE',del_cols)    
    bootdd=bootdd.drop(del_cols,axis=1)
    bootdd.fillna(0,inplace=True)
    #print("######BootDD##########\n",bootdd)
    #Some season notes are included in the table and need to be removed
    if link == 'http://survivor.wikia.com/wiki/Survivor:_Panama':
        dd.drop(dd.index[15],inplace=True)
        bootdd.drop(bootdd.index[16],inplace=True)
    if link == 'http://survivor.wikia.com/wiki/Survivor:_David_vs._Goliath':
        dd.drop(dd.index[20],inplace=True)
        bootdd.drop(bootdd.index[21],inplace=True)
    if link == 'http://survivor.wikia.com/wiki/Survivor:_Ghost_Island':
        dd.drop(dd.index[20],inplace=True)
        bootdd.drop(bootdd.index[21],inplace=True)
    if link == 'http://survivor.wikia.com/wiki/Survivor:_Game_Changers':
        dd.drop(dd.index[20],inplace=True)
        bootdd.drop(bootdd.index[21],inplace=True)
    if link == 'http://survivor.wikia.com/wiki/Survivor:_Heroes_vs._Healers_vs._Hustlers':
        dd.drop(dd.index[18],inplace=True)
        bootdd.drop(bootdd.index[19],inplace=True)
        
    dd.replace(to_replace=' ', value = '_',regex=True,inplace=True)
    return dd,bootdd

In [908]:
def return_affiliation_df(bootdd,dd):
    """
        Returns an affiliation index data frame per voting round
        input: dd - data frame of voting history
        output: affiliation_dict_list - data frame of affiliation index per voting round
    """
    global del_cols
    
    flatten = lambda l: [item for sublist in l for item in sublist]
    contestants = dd.index
    pairs = list(combinations(contestants,2))
    affiliation_dict_list = []
    
    # iterate over voting rounds
    for col_index in dd.columns:
        # get the clusters who voted for particular people
        voting_dict = dd.groupby(col_index).groups
        # if not a name, remove from the voting dict
        for i in ['','—']:
            if i in voting_dict.keys():
                voting_dict.pop(i)
        # everyone who voted in tribal council
        in_tribal = set(flatten(voting_dict.values()))
        # get all the people who received votes at tribal
        candidates = voting_dict.keys()
        candidates = [name for name in candidates if name in contestants]
                
        # get the vote counts of the people who received votes
        vote_counts = []
        for voted in candidates:
            vote_counts.append(len(voting_dict[voted]))
 
        affiliation_dict = defaultdict(int)
        # initialize all pairs of people in tribal to have affiliation value of 0
        for i in in_tribal:
            for j in in_tribal:
                if (i,j) in pairs:
                    affiliation_dict[(i,j)] = 0
        # in the case of tiebreakers, or when only 2 people are at risk, set affiliation value
        # of a pair of contestants to 1 if they voted together
        if 'None' in voting_dict.keys() or len(candidates) < 3:
            for voted in candidates:
                for i in voting_dict[voted]:
                    for j in voting_dict[voted]:
                        if (i,j) in pairs:
                            affiliation_dict[(i,j)] = 1 
            affiliation_dict_list.append(affiliation_dict)
            continue
        # in the case of split votes:
        else:
            # there are usually 3 names thrown out during split votes:
            # the 2 targets of the majority affiliation, and the target of the
            # people in the minority.

            # get who the contestants at risk voted for
            candidates_voted_for = dict()
            for voted in candidates:
                for other in candidates:
                    if voted in voting_dict[other]:
                        candidates_voted_for[voted]=other
            #print(col_index,candidates_voted_for)
                        
            # cross contains all possible 3 pairs of the contestants at risk
            cross = list(combinations(candidates,3))
            # cluster_i represents the contestants who voted for contestant i
            for (voted1, voted2, voted3) in cross:
                cluster1 = voting_dict[voted1]
                cluster2 = voting_dict[voted2]
                cluster3 = voting_dict[voted3]

                # set affiliation value of a pair of contestants to 1 if the they
                # belong to the same cluster
                for i in cluster1:
                    for j in cluster1:
                        if (i,j) in pairs:
                            affiliation_dict[(i,j)] = 1

                for i in cluster2:
                    for j in cluster2:
                        if (i,j) in pairs:
                            affiliation_dict[(i,j)] = 1

                for i in cluster3:
                    for j in cluster3:
                        if (i,j) in pairs:
                            affiliation_dict[(i,j)] = 1

                #fixes for vote steals and other advantages
                if candidates_voted_for == {'Abi-Maria': 'Stephen', 'Stephen': 'Joe'}:
                    candidates_voted_for['Joe']=None      
                if candidates_voted_for == {'Aubry': 'Tai', 'Michaela': 'Tai'}:
                    candidates_voted_for['Tai']=None
                if candidates_voted_for == {'Ben': 'Desi', 'Desi': 'Lauren', 'Joe': 'Ben'} and voted3 == 'Lauren':
                    candidates_voted_for['Lauren']=None

                    
                #print(candidates_voted_for)
                #print(voted1,voted2,voted3)
                # target_i is the vote of the ith contestant at risk
                target1 = candidates_voted_for[voted1]
                target2 = candidates_voted_for[voted2]
                target3 = candidates_voted_for[voted3]

                voteds = [voted1,voted2,voted3]
                clusters = [cluster1, cluster2, cluster3]
                targets = [target1, target2, target3]
                index_pairs = combinations([0,1,2],2)
                
                for (ix1, ix2) in index_pairs:
                    for cluster in clusters:
                        # check if more than 1 person votes for contestant ix1 and more than
                        # 1 person votes for contestant ix2, then they are candidates for the
                        # split vote.
                        if (len(clusters[ix1]) > 1) and (len(clusters[ix2]) > 1):
                            # check if the targets of contenstant ix1 and ix2 are in the same cluster
                            # if yes, then the people who voted for ix1 and ix2 did a split
                            # vote and are in an alliance
                            if (targets[ix1] in cluster) and (targets[ix2] in cluster):
                                everyone = flatten([clusters[ix1],clusters[ix2]])
                                for i in everyone:
                                    for j in everyone:
                                        if (i,j) in pairs:
                                            # assign affiliation = 1 to all who voted for 
                                            # ix1 and ix2
                                            affiliation_dict[(i,j)] = 1
                                # since ix1 and ix2 voted together, they are in a
                                # minority affiliation

                                if (voteds[ix1], voteds[ix2]) in pairs:
                                    affiliation_dict[(voteds[ix1], voteds[ix2])] = 1
                                elif (voted[ix2], voted[ix1]) in pairs:
                                    affiliation_dict[(voteds[ix2], voteds[ix1])] = 1
            affiliation_dict_list.append(affiliation_dict)
    #print(bootdd)
    #print (len(affiliation_dict_list))
    for y in range(len(affiliation_dict_list)):
        #print(bootdd.columns)
        if y not in del_cols:
    #        print(y,affiliation_dict_list[y])
            for x in affiliation_dict_list[y].keys():
    #            print(x)
                if affiliation_dict_list[y][x] == 1:
                    #print(x[0],y,x[1],bootdd[y][x[0]],bootdd[y][x[1]]) #x[0]][z],bootdd[x[1]][z])
                    newval = max(bootdd[y][x[0]],bootdd[y][x[1]])
                    bootdd[y][x[0]] = newval
                    bootdd[y][x[1]] = newval
                    #print(x,newval)
        #print(x[0],type(x))
    #print(bootdd.shape,bootdd.columns)
    return pd.DataFrame(affiliation_dict_list).T, bootdd

In [909]:
allids = pd.read_csv('/home/analytics/Survivor/ids.csv',index_col=0)
allids['Alliance_Score'] = 0
bootscore = {}
no_replace = [0,12,16,17,19,33]
doubles = [['rob','robert',4],['john', 'johnson' ,5],['lea','leann',9]
           ,['brian','brianna',11],['jon','penner',16],['matt','mattos',22]
           ,['dan', 'danielle', 12],['mark','markoski',23],['kat','katie',27]
           ,['woo','wood',28],['so','souberbielle',30],['brad','bradshaw',34]
           ,['stephanie','gonzalez',36],['taylor','CeCe',33]]
del_cols = []
first=True
#if first == True:
#    i = 21
for i in range(len(seasons)):
    season = seasons[i]
    #if season != 'Cambodia' and season != 'Game Changers' and season != 'Heroes vs. Healers vs. Hustlers':
    alliance,bootdd = plot_network(season)#,ids)
    bootdd['ID'] = int(100*(i+1))
 
    ssnid = allids["season_number"] == i+1
    ids = allids[ssnid].copy()
    count=0
    
    R = False
    P = ''
    Q = ''
    for p,q,r in doubles:
        if r == i+1:
            #print('double#########################')
            P=p.title()
            Q=q.title()
            if Q == 'Cece': Q = 'CeCe'
            R=True
            #print (P,Q,R)
    
    for x in range(len(alliance)):
        for y in range(len(ids)):
            Unique = True
            k = alliance[x][0]
            hold = k
            if i not in no_replace: k = k.replace('.','')
            #print (k, ids.iloc[y,0])
            if Q in ids.iloc[y,0] and k == P and R == True:
                Unique = False
            if i == 15:
                if 'Jones' in ids.iloc[y,0] and k == 'Jon' and R == True:
                    Unique = False
                
            if k in ids.iloc[y,0] and Unique == True:
                j = ids.iloc[y].name
                #print (j,alliance[x])
                allids.iloc[j,3] = alliance[x][1]
                ids.iloc[y,3] = alliance[x][1]
                #if i == 32: print((j,alliance[x]))
                
                #matching for boot order
                col = bootdd.columns.get_loc('ID')
                row = bootdd.index.get_loc(hold)
                bootdd.iloc[row,col] = ids.iloc[y,2]
                count+=1
                
    if len(ids)!= count:
        print("########ERROR#########")
        print(len(ids),count)
        print(alliance,ids)
    bootscore[int(i+1)] = bootdd
    bootdd.reset_index(inplace=True)
    bootdd.set_index('ID',inplace=True)
    print(bootdd.columns)
    newcols = {}
    i = 0
    for x in bootdd.columns:
        if x != 'Name':
            newcols[x] = i
            i+=1
    bootdd.rename(columns = newcols,inplace=True)
    print(bootdd.columns)
    if first==True:
        allboots = pd.DataFrame(bootdd)
        first=False
    else:
        #print("APPEND")
        allboots = allboots.append(bootdd,sort=False)
        #print(allboots)
    
allids.rename(index=str, columns={'id':'ID'},inplace=True)
allids.set_index('ID',inplace=True)        
allids.to_csv('/home/analytics/Survivor/AllianceScores.csv', index=True)
allboots.to_csv('/home/analytics/Survivor/BootStats.csv', index=True)
#allids
#bootscore.keys()


http://survivor.wikia.com/wiki/Survivor:_Borneo
Index(['Name', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_The_Australian_Outback
Index(['Name', 0, 1, 2, 5, 6, 10, 11, 12, 13, 14, 15, 16, 17], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_Africa
Index(['Name', 0, 1, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_Marquesas
found paschal
Index(['Name', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_Thailand
Index(['Name', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_The_Amazon
Index(['Name', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_Pearl_Islands
Index(['Name', 0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 16], dtype='object')
http://survivor.wikia.com/wiki/Survivor:_All-Stars
Index(['N